# Hyperspectral Burn Severity Mapping  
### From Spectral Analysis to Visualization of BC Fire K51472

This notebook will go from a raw Wyvern L2B product to a burn severity map derived from a reNDVI layer and a principal component (PC2).


In [ ]:
import rasterio
import numpy as np
from pystac import Item
from pystac.extensions.eo import EOExtension
from dateutil import parser
import earthpy.plot as ep

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [ ]:
# Load in our data (already clipped in QGIS)

AOI_Penticton = './Penticton_Recovery/Dragonette_AOI.tif'

with rasterio.open(AOI_Penticton) as src:
    image_toa_radiance = src.read()
    nodata_value = src.nodata
    image_toa_radiance[image_toa_radiance == nodata_value] = np.nan
    
    # Save these for exporting later
    transform = src.transform
    crs = src.crs # None in this instance, Dragonette-1 quirk?


### Atmospheric correction (Move from L1B to L1C)
A lot of this was "borrowed" from Wyvern's Knowledge Center here: https://github.com/Nrevyw/wyvern-public-resources/blob/main/top-of-atmosphere-processing/convert_wyvern_dragonette_data_to_top-of-atmosphere_reflectance.ipynb

This ended up not being necessary, as this projectmay have been an analysis on chlorophyl content but the project went in another direction. I only ended up comparing values within the scene and so atmospheric correction was not necessary, but couldn't have hurt and was a fun exploration of the knowledge center.

In [ ]:
item = Item.from_file("./Penticton_Recovery/wyvern_dragonette-001_20240806T172508_6b59089b.json")
dragonette_hsi_asset = item.assets["Cloud optimized GeoTiff"]
eo_bands = dragonette_hsi_asset.extra_fields["eo:bands"]
bands = EOExtension.ext(dragonette_hsi_asset).bands

collection_datetime = parser.parse(item.properties["datetime"])
collection_day_of_year = int(collection_datetime.strftime("%j"))
sun_earth_distance = 1 - (0.01672 * np.cos(np.deg2rad(0.9856 * (collection_day_of_year - 4))))
sun_elevation = item.properties["view:sun_elevation"]

# Convert the pixels in all bands to top-of-atmosphere reflectance
image_toa_reflectance = np.empty(image_toa_radiance.shape, dtype=np.float32)

for i in range(len(bands)):
    image_toa_reflectance[i, :, :] = (
        image_toa_radiance[i, :, :] * np.pi * (sun_earth_distance**2)
        ) / (bands[i].solar_illumination * np.sin(np.deg2rad(sun_elevation)))
    
# Plot color infrared RGB images
red_plot_band = 22
green_plot_band = 11
blue_plot_band = 4


fig, ax = plt.subplots(1, 2, figsize=(8, 8))

ep.plot_rgb(
    image_toa_radiance,
    rgb=(red_plot_band, green_plot_band, blue_plot_band),
    ax = ax[0],
    stretch = True,
    title="Color Infrared Top-of-Atmosphere Radiance"    
)

ep.plot_rgb(
    image_toa_reflectance,
    rgb=(red_plot_band, green_plot_band, blue_plot_band),
    ax = ax[1],
    stretch = True,
    title="Color Infrared Top-of-Atmosphere Reflectance"    
)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate reNDVI & save raster
# reNDVI = (NIR - RedEdge) / (NIR + RedEdge)

# Define band indices for Red-edge and NIR
red_edge = image_toa_reflectance[17, :, :]
nir = image_toa_reflectance[20, :, :]


reNDVI = (nir - red_edge) / (nir + red_edge)
reNDVI[np.isnan(reNDVI)] = np.nan

# Export reNDVI
output_file = './Penticton_Recovery/reNDVI.tif'
height, width = red_edge.shape
with rasterio.open(
    output_file,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=rasterio.float32,
    crs=crs,
    transform=transform,
    nodata=np.nan
) as dst:
    dst.write(reNDVI.astype(rasterio.float32), 1)
    print("reNDVI calculation complete! Saved to:", output_file)
    


## Running PCA Analysis and Analyzing Scree Plot & Eigenvalues

In [ ]:
# Now let's do a PCA analysis to reduce dimensionality of our data.
len_bands, height, width = image_toa_reflectance.shape
X = image_toa_reflectance.reshape(len_bands, -1).T

pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)

# Eigenvalues & vectors
eigenvalues = pca.explained_variance_
explained = pca.explained_variance_ratio_
loadings = pca.components_ 

# Number of principle components
n_pcs = loadings.shape[0]

In [ ]:
len_bands, height, width = image_toa_reflectance.shape
X = image_toa_reflectance.reshape(len_bands, -1).T

pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)

# Eigenvalues & vectors
eigenvalues = pca.explained_variance_
explained = pca.explained_variance_ratio_
loadings = pca.components_ 
n_pcs = loadings.shape[0]

# Make a Scree plot to visualize each PC's contribution to overall variance.
plt.figure(figsize=(6,4))
plt.plot(np.arange(1, len(eigenvalues)+1), explained*100, 'o-')
plt.xlabel("Principal Component")
plt.ylabel("Variance Explained (%)")
plt.title("Scree Plot - Wyvern Hyperspectral PCA")
plt.grid(True)
plt.show()

In [ ]:
# Get bands center wavelength and convert to nm for PCA plot
band_collector = []
for band in eo_bands:
    band_collector.append(int(band["center_wavelength"] * 1000)) # Convert from micron to nm
    
wavelengths = sorted(band_collector)
print(f"Sensor center wavelengths(nm): \n{wavelengths}")    

In [ ]:
plt.figure(figsize=(15,5))

# Plot the first X PCs' loadings
for i in range(3):
    plt.plot(wavelengths,
             loadings[i],
             marker='o',
             markersize=3,
             linewidth=1.2,
             label=f"PC{i+1} | Eigenvalue: {(explained[i]*100):.2f}%"
    )

plt.xlabel("Wavelength (nm)", labelpad=15)
plt.xticks(wavelengths, rotation=90)
plt.ylabel("Loading Weight")
plt.title("PCA Loadings vs. Wavelength")
plt.grid(True, linestyle="--", alpha=0.4)

# Highlight spectra of interest
plt.axvspan(680, 750, color='orange', alpha=0.15, label='Red edge transition')


plt.legend()
plt.tight_layout()
plt.show()

## Here we run our k-means clustering to create the final burn severity map using our reNDVI and PC2 layers

In [ ]:
# Get clipped images
with rasterio.open('./Penticton_Recovery/reNDVI_Clipped.tif') as reNDVI_file:
    reNDVI = reNDVI_file.read(1) 
    height, width = reNDVI.shape
    # Save these for exporting later
    profile = reNDVI_file.profile 
    transform = reNDVI_file.transform
    crs = reNDVI_file.crs
    dtype = reNDVI.dtype
    
with rasterio.open('./Penticton_Recovery/PC2_Clipped.tif') as PC2_file:
    PC2 = PC2_file.read(1)
    
    
    
# Flatten layers
reNDVI_flat = reNDVI.flatten()
PC2_flat   = PC2.flatten()

# Stack into (N, 2)
X = np.vstack([reNDVI_flat, PC2_flat]).T

# Remove NaNs
mask = ~np.any(np.isnan(X), axis=1)
X_clean = X[mask]

n_clusters = 6

kmeans = KMeans(n_clusters = n_clusters, n_init = 'auto')
kmeans.fit(X_clean)

labels_clean = kmeans.labels_

# Create full label array (with NaNs in no-data areas)
labels_full = np.full(reNDVI_flat.shape, fill_value=np.nan)
labels_full[mask] = labels_clean

# Reshape back to original raster shape
labels_image = labels_full.reshape(reNDVI.shape)

severity_order = []

# Rank clusters based on reNDVI (better measure of vegetation loss)
for cluster in range(n_clusters):
    mean = np.nanmean(reNDVI_flat[labels_full == cluster])
    severity_order.append((cluster, mean))

# Sort severity by reNDVI (vegitation loss)
severity_order_sorted = sorted(severity_order, key=lambda x: x[1])

# Map cluster values
cluster_to_severity = {cluster: n_clusters - 1 - rank for  rank, (cluster, _) in enumerate(severity_order_sorted)}

# Apply re-labeling
severity_map = np.copy(labels_image)
for old, new in cluster_to_severity.items():
    severity_map[labels_image == old] = new
    
# Visualize our K-clustered burn severity map
n_classes = int(np.nanmax(severity_map)) + 1
# Give it a colormap
cmap = plt.cm.YlOrRd 
cmap = ListedColormap(cmap(np.linspace(0, 1, n_classes)))

# Set boundaries so each integer label maps to a color
bounds = np.arange(-0.5, n_classes, 1)
norm = BoundaryNorm(bounds, ncolors=n_classes)

# Plot
plt.figure(figsize=(10, 6))
im = plt.imshow(severity_map, cmap=cmap)
cbar = plt.colorbar(im, ticks=np.arange(n_classes))
cbar.set_label("Burn Severity")
cbar.ax.set_yticklabels([str(i) for i in range(n_classes)])  # or custom labels
plt.title("Burn Severity Map")
plt.show()

# Export
filename = f"./Penticton_PCA/Burn_Severity_Map.tif"
with rasterio.open(
    filename,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=dtype,
    crs=crs,
    transform=transform
) as dst:
    dst.write(severity_map, 1)